In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.models as models
import numpy as np
import pandas as pd
import cv2
import pydicom
from sklearn.utils import shuffle
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from PIL import Image

In [ ]:
df = pd.read_csv('/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/sample_submission.csv')

In [ ]:
# class Dataset(torch.utils.data.Dataset):
#     def __init__(self, df, transforms):
#         self.transforms = transforms
#         self.df = df

#     def __getitem__(self, idx):

#         img_path = '/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/test/' \
#                     + self.df.iloc[idx]['image_id'] + '.dicom'
#         dicom = pydicom.dcmread(img_path)
#         dicom.BitsStored = 16
#         img = dicom.pixel_array.astype('float32')
#         if dicom.PhotometricInterpretation == "MONOCHROME1":
#             img = np.amax(img) - img
#         intercept = int(dicom.RescaleIntercept) if "RescaleIntercept" in dicom else 0.0
#         slope = float(dicom.RescaleSlope) if "RescaleSlope" in dicom else 1.0
#         img = (img * slope).astype('int16') + intercept
#         img = np.stack([img, img, img])
#         img = (img - img.min()) / (img.max() - img.min()) * 255.
#         img = img.transpose(1, 2, 0)

#         img = self.transforms(image=img)['image']
#         return img

#     def __len__(self):
#         return len(self.df)



In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, df, transforms):
        self.transforms = transforms
        self.df = df

    def __getitem__(self, idx):

        img_path = '/kaggle/input/vinbigdata-test-jpg/' \
                    + self.df.iloc[idx]['image_id'] + '.jpg'
        img = np.array(Image.open(img_path))
        img = self.transforms(image=img)['image']
        return img

    def __len__(self):
        return len(self.df)



In [ ]:
test_transforms = A.Compose([
                    A.Normalize(mean=(0, 0, 0), std=(1, 1, 1), max_pixel_value=255.0, p=1.0),
                    ToTensorV2(p=1.0)
                    ])

In [ ]:
def collate_fn(batch):
    images = [image for (image) in batch]
    return images


In [ ]:
test_dataset = Dataset(transforms=test_transforms, df=df)
test_dataloader = torch.utils.data.DataLoader(
                                         dataset=test_dataset,
                                         batch_size=16,
                                         shuffle=False,
                                         collate_fn=collate_fn,
                                         num_workers=2
                                         )

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False)
num_classes = 15
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
model = model.to(device)
model = nn.DataParallel(model)
model.load_state_dict(torch.load('/kaggle/input/fasterrcnn-model/model_28.pth'))
model = model.module

In [ ]:
model = model.eval()

In [ ]:
y_preds = []
for b, x in enumerate(test_dataloader):
    print(b)
    x = [xi.to(device) for xi in x]
    with torch.no_grad():
        output = model(x)
    for i, output_i in enumerate(output):
        scores = output_i['scores']
        labels = output_i['labels']
        boxes = output_i['boxes']
        labels = labels[scores > 0.5]
        boxes = boxes[scores > 0.5]
        scores = scores[scores > 0.5]
        if len(labels) == 0 or labels[0] == 0:
            y_preds.append('14 1 0 0 1 1')
        else:
            y_pred = []
            for j in range(len(labels)):
                box = list(map(str, boxes[j].cpu().numpy()))
                y_pred += [f"{labels[j].item()-1} {scores[j].item()} {' '.join(box)}"]
            y_preds.append(' '.join(y_pred))


In [ ]:
df['PredictionString'] = y_preds
df.to_csv('submission.csv', index=False)

In [ ]:
df